<a href="https://colab.research.google.com/github/RaviKcse08/datascience_projs/blob/main/DC_Intro_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from pprint import pprint

In [20]:
sd = pd.read_csv('bank_marketing.csv')
#print(sd.shape)
sd.head()
dframe = sd.dropna()
dframe.head()




,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,,pdays,previous,putcome,y
0,51,management,married,tertiary,no,6530,yes,no,unknown,5,may,91,1,-1,0,unknown,no
1,27,unemployed,married,primary,no,7459,yes,no,unknown,9,may,407,2,-1,0,unknown,no
2,59,management,divorced,primary,no,13308,yes,no,unknown,13,may,131,1,-1,0,unknown,no
3,48,technician,married,secondary,no,7373,yes,no,unknown,14,may,126,1,-1,0,unknown,no
4,51,blue-collar,married,secondary,no,7180,yes,no,unknown,14,may,927,1,-1,0,unknown,yes


In [40]:
#categorize classes
df = dframe.rename(columns={"y": "label"})


##Select Xs - age, job, marital, education, balance)
df = df[['age','job','marital','education','balance','label']].head(500)
df.shape

(500, 6)

In [48]:
#def get_count(values):
#    return len(values)
#df.groupby("job").agg(get_count)
df = df[['age','job','marital','education','balance','label']].head(500)
#df.map({'management': 101, 'technician': '101'})

In [49]:
train_df, test_df = train_test_split(df, test_size=.20)
train_df.shape
#data[:5]

(400, 6)

In [50]:
#helper function to check uncertainity
def check_entropy(data):
    ##Y in the marketing data - label column
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

#return true if label_column has only 1 value otherwise false
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [51]:
def classify_data(data):
    ##Y in the marketing data
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]    
    return classification

In [30]:
def get_potential_splits(data):
    
    potential_splits = {}
    n_records, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [53]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]
    if isinstance(split_value, str):
          data_below = data[split_column_values == split_value]
          data_above = data[split_column_values !=  split_value]
    else:
          data_below = data[split_column_values <= split_value]
          data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

In [54]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [55]:
def decision_tree_algorithm(df, counter=0):
    
    # data preparations
    if counter == 0:
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if check_entropy(data):
        classification = classify_data(data)
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        question = "{} <= {}".format(split_column, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter)
        no_answer = decision_tree_algorithm(data_above, counter)
        
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
        
        return sub_tree

In [57]:
tree = decision_tree_algorithm(train_df)
pprint(tree)

TypeError: ignored